# 📰 AM Source Compiler

AM Source compiler merges research sheets based on `source names` and `domain url`. The output is a master list of the sources in the country researched.


In [3]:
#SELCT YOUR RESEARCH SHEET"S URL
URL = ''

## Google Authenticator

AM Source Compiler works with your Google research sheet. Input the URL to your sheet and authenticate yourself.

In [4]:
import pandas as pd
from google.colab import auth
import gspread
from google.auth import default
from gspread_dataframe import set_with_dataframe
import re
import numpy as np


Google Authenticator opens in a new window. Give your permisssion to AM Source Compiler to access your sheets.

In [ ]:
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

## Load Research Sheet

Select releveant research doc and load sheets.

In [ ]:
sh = gc.open_by_url(URL)

In [2]:
# Load data
def open_workbook(workbook):
  ws = sh.worksheet(workbook)
  rows = ws.get_all_values()
  return pd.DataFrame.from_records(rows[1:], columns = rows[0])

Select the sheets that you want to merge into our master list.

In [ ]:
#Get workbooks=
comply = open_workbook('comply')
worldnews = open_workbook('world_newspapers')
wikipedia = open_workbook('wikipedia')
press_council = open_workbook("press council of ireland")
mediainfo = open_workbook("mediainfo")

## Prepare domains

URLs come in different formats. To improve matching, we ensure only take what's after 'www.'

In [ ]:
#Clean url
extract_domain = lambda url: re.sub(r'^(https?://)?(www\.)?|(\/)+$', '', url)

wikipedia['domain'] = wikipedia['domain'].apply(lambda x: extract_domain(x))
comply['domain'] = comply['domain'].apply(lambda x: extract_domain(x))
worldnews['domain'] = worldnews['domain'].apply(lambda x: extract_domain(x))
mediainfo['domain'] = mediainfo['domain'].apply(lambda x: extract_domain(x))
press_council['domain'] = press_council['domain'].apply(lambda x: extract_domain(x))

## Merge and group into Main list

In [ ]:
#Concat sheets
concat_data = pd.concat([wikipedia, comply, worldnews, mediainfo, press_council]).reset_index()

In [ ]:
# Concat sheets and group by name with aggregated rows as list
merged_df = concat_data.groupby(['name', 'domain']).agg(tuple).applymap(list).reset_index()
merged_df = concat_data.groupby('name').agg(tuple).applymap(list).reset_index()
merged_df = merged_df.sort_values(by='name', ascending=True)

## Data Cleaning

Ensure that there's only one unique value in provider and taxonomy fields for easier filtering in sheet. *italicized text*

In [ ]:
merged_df['provider'] = merged_df['provider'].apply(lambda x: set(x))
merged_df['taxonomy'] = merged_df['taxonomy'].apply(lambda x: set(x))

In [ ]:
def stringify(lst):
  lst = [x for x in lst if not (isinstance(x, float) and np.isnan(x))]
  lst = [x for x in lst if x !=""] #Remove where empty string ie if comply does not have taxonomy ""
  unique = set(lst)
  return ", ".join(str(item) for item in unique)


merged_df['taxonomy'] = merged_df['taxonomy'].apply(lambda x: stringify(x))
merged_df['provider'] = merged_df['provider'].apply(lambda x: stringify(x))
merged_df['frequency'] = merged_df['frequency'].apply(lambda x: stringify(x))
merged_df['type'] = merged_df['type'].apply(lambda x: stringify(x))




## Write to file

In [ ]:
if 'main' in [worksheet.title for worksheet in sh.worksheets()]:
  worksheet = sh.worksheet('main')
  set_with_dataframe(worksheet, merged_df)
else:
    sh.add_worksheet(title='main', rows=100, cols=30)
    worksheet = sh.worksheet('main')
    set_with_dataframe(worksheet, merged_df)
